In [25]:
import argparse
import os
import time
import sys
#被import的子函数，需要以.py的形式存在，需要执行的才使用.ipynb
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms  # for data preprocess
import flow_transforms
import models
import datasets
from multiscaleloss import multiscaleEPE, realEPE
import datetime
from torch.utils.tensorboard import SummaryWriter  # use tensorboard to visualize training process
from util import flow2rgb, AverageMeter, save_checkpoint


In [30]:
model_name = "PWCNet"
dataset_names = sorted(name for name in datasets.__all__)

# create a ArgumentParser object
parser = argparse.ArgumentParser(description='PyTorch PWCNet Training on several datasets',
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# add arguments
parser.add_argument('data', metavar='Dir',
                    help='path to dataset')
parser.add_argument('--dataset', metavar='DATASET', default='mpi_sintel_clean',
                    choices=dataset_names,
                    help='dataset type : ' +
                         ' | '.join(dataset_names))
group = parser.add_mutually_exclusive_group()  # create exclusive paras
group.add_argument('-s', '--split-file', default=None, type=str,
                   help='test-val split file')
group.add_argument('--split-value', default=0.8, type=float,
                   help='test-val split proportion between 0 (only test) and 1 (only train), '
                        'will be overwritten if a split file is set')
parser.add_argument(
    "--split-seed",
    type=int,
    default=None,
    help="Seed the train-val split to enforce reproducibility (consistent restart too)",
)
parser.add_argument('--arch', '-a', metavar='ARCH', default='pwc_dc_net',
                    choices=model_name,
                    help='model architecture, overwritten if pretrained is specified: ' +
                         ' | '.join(model_name))
parser.add_argument('--solver', default='adam', choices=['adam', 'sgd'],
                    help='solver algorithms')
parser.add_argument('-j', '--workers', default=8, type=int, metavar='N',
                    help='number of data loading workers')
parser.add_argument('--epochs', default=300, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--epoch-size', default=1000, type=int, metavar='N',
                    help='manual epoch size (will match dataset size if set to 0)')
parser.add_argument('-b', '--batch-size', default=8, type=int,
                    metavar='N', help='mini-batch size')
parser.add_argument('--lr', '--learning-rate', default=0.0001, type=float,
                    metavar='LR', help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M',
                    help='momentum for sgd, alpha parameter for adam')
parser.add_argument('--beta', default=0.999, type=float, metavar='M',
                    help='beta parameter for adam')
parser.add_argument('--weight-decay', '--wd', default=4e-4, type=float,
                    metavar='W', help='weight decay')
parser.add_argument('--bias-decay', default=0, type=float,
                    metavar='B', help='bias decay')
parser.add_argument('--multiscale-weights', '-w', default=[0.005, 0.01, 0.02, 0.08, 0.32], type=float, nargs=5,
                    help='training weight for each scale, from highest resolution (flow2) to lowest (flow6)',
                    metavar=('W2', 'W3', 'W4', 'W5', 'W6'))
parser.add_argument('--sparse', action='store_true',
                    help='look for NaNs in target flow when computing EPE, avoid if flow is garantied to be dense,'
                         'automatically seleted when choosing a KITTIdataset')
parser.add_argument('--print-freq', '-p', default=10, type=int,
                    metavar='N', help='print frequency')
parser.add_argument('-e', '--evaluate', dest='evaluate', action='store_true',
                    help='evaluate model on validation set')
parser.add_argument('--pretrained', dest='pretrained', default='F:\Python_project\PWC-Net-main\PyTorch\pwc_net.pth.tar',
                    help='path to pre-trained model')
parser.add_argument('--no-date', action='store_true',
                    help='don\'t append date timestamp to folder')
parser.add_argument('--div-flow', default=20,
                    help='value by which flow will be divided. Original value is 20 but 1 with batchNorm gives good results')
parser.add_argument('--milestones', default=[100, 150, 200], metavar='N', nargs='*',
                    help='epochs at which learning rate is divided by 2')




_StoreAction(option_strings=['--milestones'], dest='milestones', nargs='*', const=None, default=[100, 150, 200], type=None, choices=None, required=False, help='epochs at which learning rate is divided by 2', metavar='N')

In [31]:
global args, best_EPE
best_EPE = -1
sys.args = [ "data","D:/My_Jupyter Notebook/PWC-Net-main/PyTorch/MPI-Sintel-complete/training/"] 
#sys.args = [ "--data","D:/My_Jupyter Notebook/PWC-Net-main/PyTorch/MPI-Sintel-complete/training/"] 这样是行不通的
args = parser.parse_args()
n_iter = int(args.start_epoch)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




In [32]:
def main():
    global args, best_EPE

    args = parser.parse_args()
    save_path = '{},{},{}epochs{},b{},lr{}'.format(
        args.arch,
        args.solver,
        args.epochs,
        ',epochSize' + str(args.epoch_size) if args.epoch_size > 0 else '',
        args.batch_size,
        args.lr)
    if not args.no_date:
        timestamp = datetime.datetime.now().strftime("%m-%d-%H-%M")
        save_path = os.path.join(timestamp, save_path)
    save_path = os.path.join(args.dataset, save_path)
    print('=> will save everything to {}'.format(save_path))
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    if args.split_seed is not None:
        np.random.seed(args.split_seed)

    train_writer = SummaryWriter(os.path.join(save_path, 'train'))
    test_writer = SummaryWriter(os.path.join(save_path, 'test'))
    output_writers = []
    for i in range(3):
        output_writers.append(SummaryWriter(os.path.join(save_path, 'test', str(i))))

    # Data loading code
    input_transform = transforms.Compose([  # preprocess the training set
        flow_transforms.ArrayToTensor(),  # turn the GT flow array to tensor
        transforms.Normalize(mean=[0, 0, 0], std=[255, 255, 255]),  # normalization->converge speeding
        transforms.Normalize(mean=[0.45, 0.432, 0.411], std=[1, 1, 1])  # zero-centered->converge speeding
    ])
    '''output flow hs only two dimension: u and v, both will be divided by 20'''
    target_transform = transforms.Compose([  # preprocess the testing set
        flow_transforms.ArrayToTensor(),  # turn the output flow array to tensor
        transforms.Normalize(mean=[0, 0], std=[args.div_flow, args.div_flow])
    ])

    if 'KITTI' in args.dataset:
        args.sparse = True
    if args.sparse:
        co_transform = flow_transforms.Compose([  # data augmentation
            flow_transforms.RandomCrop((320, 448)),
            flow_transforms.RandomVerticalFlip(),
            flow_transforms.RandomHorizontalFlip()
        ])
    else:
        co_transform = flow_transforms.Compose([
            flow_transforms.RandomTranslate(10),
            flow_transforms.RandomRotate(10, 5),
            flow_transforms.RandomCrop((320, 448)),
            flow_transforms.RandomVerticalFlip(),
            flow_transforms.RandomHorizontalFlip()
        ])

    print("=> fetching img pairs in '{}'".format(args.data))
    train_set, test_set = datasets.__dict__[args.dataset](  # split into training and testing set

        args.data,
        transform=input_transform,
        target_transform=target_transform,
        co_transform=co_transform,
        split=args.split_file if args.split_file else args.split_value
    )
    print('{} samples found, {} train samples and {} test samples '.format(len(test_set) + len(train_set),
                                                                           len(train_set),
                                                                           len(test_set)))

    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=args.batch_size,
        num_workers=args.workers, pin_memory=True, shuffle=True)
    val_loader = torch.utils.data.DataLoader(
        test_set, batch_size=args.batch_size,
        num_workers=args.workers, pin_memory=True, shuffle=False)

    # create model
    pwc_pretrained_model_fn = './pwc_net.pth.tar';
    if args.pretrained:  # start from pre-trained model
        # network_data = torch.load(args.pretrained)  # load pre-trained paras
        # args.arch = network_data['arch']
        model = models.PWCNet.pwc_dc_net(pwc_pretrained_model_fn)
        args.arch = 'pwc_dc_net'
        print("=> using pre-trained model '{}'".format(args.arch))
    else:  # start from scratch
        network_data = None
        print("=> creating model '{}'".format(args.arch))

   #  model = models.__dict__[args.arch](network_data).to(device)

    assert (args.solver in ['adam', 'sgd'])
    print('=> setting {} solver'.format(args.solver))
    param_groups = [{'params': model.bias_parameters(), 'weight_decay': args.bias_decay},
                    {'params': model.weight_parameters(), 'weight_decay': args.weight_decay}]

    if device.type == "cuda":
        model = torch.nn.DataParallel(model).cuda()
        cudnn.benchmark = True

    # optimizer
    if args.solver == 'adam':
        optimizer = torch.optim.Adam(param_groups, args.lr,
                                     betas=(args.momentum, args.beta))
    elif args.solver == 'sgd':
        optimizer = torch.optim.SGD(param_groups, args.lr,
                                    momentum=args.momentum)

    if args.evaluate:
        best_EPE = validate(val_loader, model, 0, output_writers)
        return

    # using multistep learning rate
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=args.milestones, gamma=0.5)

    for epoch in range(args.start_epoch, args.epochs):
        scheduler.step()

        # TRAIN for one epoch
        train_loss, train_EPE = train(train_loader, model, optimizer, epoch, train_writer)  # computing loss and EPE
        train_writer.add_scalar('mean EPE', train_EPE, epoch)

        # EVALUATE on validation set

        with torch.no_grad():
            EPE = validate(val_loader, model, epoch, output_writers)
        test_writer.add_scalar('mean EPE', EPE, epoch)

        if best_EPE < 0:
            best_EPE = EPE

        is_best = EPE < best_EPE
        best_EPE = min(EPE, best_EPE)  # update best EPE
        save_checkpoint({  # save best paras
            'epoch': epoch + 1,
            'arch': args.arch,
            'state_dict': model.module.state_dict(),
            'best_EPE': best_EPE,
            'div_flow': args.div_flow
        }, is_best, save_path)


def train(train_loader, model, optimizer, epoch, train_writer):
    global n_iter, args
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    pwc_EPES = AverageMeter()

    epoch_size = len(train_loader) if args.epoch_size == 0 else min(len(train_loader), args.epoch_size)

    # switch to train mode
    model.train()

    end = time.time()

    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        target = target.to(device)
        input = torch.cat(input, 1).to(device)

        # compute output
        output = model(input)
        if args.sparse:
            # Since Target pooling is not very precise when sparse,
            # take the highest resolution prediction and upsample it instead of downsampling target
            h, w = target.size()[-2:]
            output = [F.interpolate(output[0], (h, w)), *output[1:]]

        loss = multiscaleEPE(output, target, weights=args.multiscale_weights, sparse=args.sparse)
        pwc_EPE = args.div_flow * realEPE(output[0], target, sparse=args.sparse)
        # record loss and EPE
        losses.update(loss.item(), target.size(0))
        train_writer.add_scalar('train_loss', loss.item(), n_iter)
        pwc_EPES.update(pwc_EPE.item(), target.size(0))

        # compute gradient and do optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t Time {3}\t Data {4}\t Loss {5}\t EPE {6}'
                  .format(epoch, i, epoch_size, batch_time,
                          data_time, losses, pwc_EPES))
        n_iter += 1
        if i >= epoch_size:
            break

    return losses.avg, pwc_EPES.avg


def validate(val_loader, model, epoch, output_writers):
    global args

    batch_time = AverageMeter()
    pwc_EPEs = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.to(device)
        input = torch.cat(input, 1).to(device)

        # compute output
        output = model(input)
        pwc_EPE = args.div_flow * realEPE(output, target, sparse=args.sparse)
        # record EPE
        pwc_EPEs.update(pwc_EPE.item(), target.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i < len(output_writers):  # log first output of the first batches
            if epoch == args.start_epoch:
                mean_values = torch.tensor([0.45, 0.432, 0.411], dtype=input.dtype).view(3, 1, 1)
                output_writers[i].add_image('GroundTruth', flow2rgb(args.div_flow * target[0], max_value=10), 0)
                output_writers[i].add_image('Inputs', (input[0, :3].cpu() + mean_values).clamp(0, 1), 0)
                output_writers[i].add_image('Inputs', (input[0, 3:].cpu() + mean_values).clamp(0, 1), 1)
            output_writers[i].add_image('PWCNet Outputs', flow2rgb(args.div_flow * output[0], max_value=10), epoch)

        if i % args.print_freq == 0:
            print('Test: [{0}/{1}]\t Time {2}\t EPE {3}'
                  .format(i, len(val_loader), batch_time, pwc_EPEs))

    print(' * EPE {:.3f}'.format(pwc_EPEs.avg))

    return pwc_EPEs.avg


if __name__ == '__main__':
    main()

=> will save everything to mpi_sintel_clean\11-27-21-25\pwc_dc_net,adam,300epochs,epochSize1000,b8,lr0.0001
=> fetching img pairs in 'D:/My_Jupyter Notebook/PWC-Net-main/PyTorch/MPI-Sintel-complete/training/'
1041 samples found, 809 train samples and 232 test samples 
=> using pre-trained model 'pwc_dc_net'
=> setting adam solver


D:\py\Anaconda3\envs\sdsd_torch\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
D:\py\Anaconda3\envs\sdsd_torch\lib\site-packages\torch\nn\functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Epoch: [0][0/102]	 Time 36.014 (36.014)	 Data 22.660 (22.660)	 Loss 42.812 (42.812)	 EPE 4.532 (4.532)
Epoch: [0][10/102]	 Time 0.240 (3.477)	 Data 0.000 (2.060)	 Loss 31.186 (34.922)	 EPE 2.917 (3.412)
Epoch: [0][20/102]	 Time 0.257 (1.969)	 Data 0.118 (1.129)	 Loss 37.919 (38.615)	 EPE 3.351 (4.387)
Epoch: [0][30/102]	 Time 0.242 (1.492)	 Data 0.056 (0.862)	 Loss 41.749 (38.930)	 EPE 5.565 (4.527)
Epoch: [0][40/102]	 Time 0.241 (1.242)	 Data 0.031 (0.714)	 Loss 35.171 (37.542)	 EPE 4.544 (4.341)
Epoch: [0][50/102]	 Time 0.404 (1.137)	 Data 0.248 (0.671)	 Loss 31.132 (36.139)	 EPE 3.518 (4.076)
Epoch: [0][60/102]	 Time 0.235 (1.028)	 Data 0.000 (0.603)	 Loss 23.963 (35.616)	 EPE 1.906 (4.001)
Epoch: [0][70/102]	 Time 0.234 (0.952)	 Data 0.000 (0.558)	 Loss 29.770 (36.230)	 EPE 2.869 (4.107)
Epoch: [0][80/102]	 Time 0.245 (0.893)	 Data 0.000 (0.522)	 Loss 43.646 (37.237)	 EPE 4.144 (4.273)
Epoch: [0][90/102]	 Time 0.523 (0.870)	 Data 0.386 (0.518)	 Loss 30.457 (36.783)	 EPE 1.769 (4.18

Epoch: [6][60/102]	 Time 0.247 (0.930)	 Data 0.000 (0.683)	 Loss 25.190 (32.888)	 EPE 2.967 (3.606)
Epoch: [6][70/102]	 Time 0.254 (0.879)	 Data 0.000 (0.633)	 Loss 49.016 (32.302)	 EPE 5.485 (3.443)
Epoch: [6][80/102]	 Time 3.749 (0.885)	 Data 3.571 (0.639)	 Loss 14.331 (31.651)	 EPE 0.902 (3.318)
Epoch: [6][90/102]	 Time 0.247 (0.852)	 Data 0.000 (0.606)	 Loss 92.040 (31.923)	 EPE 14.295 (3.363)
Epoch: [6][100/102]	 Time 0.230 (0.818)	 Data 0.000 (0.574)	 Loss 24.372 (31.124)	 EPE 1.681 (3.226)
Test: [0/29]	 Time 5.938 (5.938)	 EPE 0.722 (0.722)
Test: [10/29]	 Time 0.271 (0.808)	 EPE 1.034 (2.647)
Test: [20/29]	 Time 0.221 (0.540)	 EPE 0.741 (3.232)
 * EPE 2.717
Epoch: [7][0/102]	 Time 20.234 (20.234)	 Data 20.029 (20.029)	 Loss 29.541 (29.541)	 EPE 2.575 (2.575)
Epoch: [7][10/102]	 Time 0.257 (2.385)	 Data 0.000 (2.138)	 Loss 44.049 (28.218)	 EPE 6.657 (2.787)
Epoch: [7][20/102]	 Time 0.253 (1.541)	 Data 0.001 (1.289)	 Loss 46.297 (28.936)	 EPE 5.709 (2.775)
Epoch: [7][30/102]	 Time

Test: [10/29]	 Time 0.312 (0.760)	 EPE 1.008 (3.993)
Test: [20/29]	 Time 0.248 (0.528)	 EPE 0.737 (3.988)
 * EPE 3.263
Epoch: [13][0/102]	 Time 20.925 (20.925)	 Data 20.606 (20.606)	 Loss 20.148 (20.148)	 EPE 1.880 (1.880)
Epoch: [13][10/102]	 Time 0.302 (2.474)	 Data 0.000 (2.187)	 Loss 23.056 (28.336)	 EPE 1.837 (2.621)
Epoch: [13][20/102]	 Time 0.334 (1.598)	 Data 0.000 (1.311)	 Loss 27.308 (31.222)	 EPE 2.456 (3.053)
Epoch: [13][30/102]	 Time 0.307 (1.299)	 Data 0.000 (1.008)	 Loss 18.610 (30.586)	 EPE 1.178 (2.907)
Epoch: [13][40/102]	 Time 3.505 (1.214)	 Data 3.302 (0.929)	 Loss 25.374 (31.096)	 EPE 1.565 (3.061)
Epoch: [13][50/102]	 Time 0.295 (1.103)	 Data 0.001 (0.823)	 Loss 28.620 (32.977)	 EPE 2.370 (3.356)
Epoch: [13][60/102]	 Time 0.357 (1.027)	 Data 0.000 (0.746)	 Loss 13.988 (31.991)	 EPE 0.719 (3.206)
Epoch: [13][70/102]	 Time 0.278 (0.969)	 Data 0.000 (0.687)	 Loss 15.302 (32.073)	 EPE 0.810 (3.273)
Epoch: [13][80/102]	 Time 3.266 (0.970)	 Data 3.052 (0.687)	 Loss 30.6

...

Epoch: [274][60/102]	 Time 0.257 (1.047)	 Data 0.001 (0.788)	 Loss 51.088 (24.163)	 EPE 6.410 (1.950)
Epoch: [274][70/102]	 Time 0.245 (0.992)	 Data 0.001 (0.734)	 Loss 18.340 (24.214)	 EPE 1.085 (1.953)
Epoch: [274][80/102]	 Time 4.282 (0.999)	 Data 3.945 (0.741)	 Loss 12.638 (23.678)	 EPE 0.730 (1.878)
Epoch: [274][90/102]	 Time 0.231 (0.960)	 Data 0.000 (0.702)	 Loss 14.538 (23.811)	 EPE 0.813 (1.905)
Epoch: [274][100/102]	 Time 0.228 (0.920)	 Data 0.001 (0.664)	 Loss 17.790 (23.486)	 EPE 0.885 (1.866)
Test: [0/29]	 Time 5.545 (5.545)	 EPE 0.673 (0.673)
Test: [10/29]	 Time 0.276 (0.768)	 EPE 0.886 (3.143)
Test: [20/29]	 Time 0.242 (0.529)	 EPE 0.670 (3.245)
 * EPE 2.661
Epoch: [275][0/102]	 Time 21.368 (21.368)	 Data 21.016 (21.016)	 Loss 19.948 (19.948)	 EPE 1.750 (1.750)
Epoch: [275][10/102]	 Time 0.241 (2.515)	 Data 0.000 (2.256)	 Loss 20.460 (26.794)	 EPE 1.260 (2.387)
Epoch: [275][20/102]	 Time 0.243 (1.627)	 Data 0.000 (1.367)	 Loss 27.930 (24.543)	 EPE 1.706 (1.961)
Epoch: [2

Test: [0/29]	 Time 5.520 (5.520)	 EPE 0.657 (0.657)
Test: [10/29]	 Time 0.283 (0.765)	 EPE 0.887 (3.106)
Test: [20/29]	 Time 0.238 (0.530)	 EPE 0.653 (3.200)
 * EPE 2.625
Epoch: [281][0/102]	 Time 21.284 (21.284)	 Data 20.884 (20.884)	 Loss 24.548 (24.548)	 EPE 1.630 (1.630)
Epoch: [281][10/102]	 Time 0.246 (2.509)	 Data 0.000 (2.242)	 Loss 24.512 (27.019)	 EPE 1.899 (2.318)
Epoch: [281][20/102]	 Time 0.247 (1.628)	 Data 0.001 (1.366)	 Loss 12.085 (23.148)	 EPE 0.392 (1.771)
Epoch: [281][30/102]	 Time 0.249 (1.312)	 Data 0.000 (1.051)	 Loss 16.355 (21.230)	 EPE 1.121 (1.545)
Epoch: [281][40/102]	 Time 4.062 (1.244)	 Data 3.740 (0.980)	 Loss 18.573 (21.367)	 EPE 1.775 (1.604)
Epoch: [281][50/102]	 Time 0.242 (1.126)	 Data 0.001 (0.861)	 Loss 19.840 (21.684)	 EPE 1.592 (1.673)
Epoch: [281][60/102]	 Time 0.246 (1.049)	 Data 0.000 (0.785)	 Loss 25.933 (22.079)	 EPE 1.623 (1.717)
Epoch: [281][70/102]	 Time 0.239 (0.993)	 Data 0.001 (0.729)	 Loss 20.215 (22.398)	 EPE 1.363 (1.751)
Epoch: [28

Epoch: [287][30/102]	 Time 0.234 (1.320)	 Data 0.001 (1.054)	 Loss 23.588 (25.601)	 EPE 2.020 (2.075)
Epoch: [287][40/102]	 Time 4.120 (1.252)	 Data 3.780 (0.987)	 Loss 18.660 (24.270)	 EPE 1.245 (1.947)
Epoch: [287][50/102]	 Time 0.243 (1.131)	 Data 0.000 (0.868)	 Loss 26.163 (23.920)	 EPE 2.098 (1.903)
Epoch: [287][60/102]	 Time 0.247 (1.054)	 Data 0.000 (0.794)	 Loss 39.356 (23.986)	 EPE 4.836 (1.933)
Epoch: [287][70/102]	 Time 0.239 (1.001)	 Data 0.000 (0.741)	 Loss 11.942 (24.075)	 EPE 0.709 (1.964)
Epoch: [287][80/102]	 Time 4.075 (1.005)	 Data 3.761 (0.745)	 Loss 19.827 (23.766)	 EPE 1.371 (1.905)
Epoch: [287][90/102]	 Time 0.238 (0.968)	 Data 0.000 (0.708)	 Loss 25.255 (23.798)	 EPE 1.691 (1.892)
Epoch: [287][100/102]	 Time 0.223 (0.927)	 Data 0.000 (0.672)	 Loss 32.423 (23.913)	 EPE 2.703 (1.926)
Test: [0/29]	 Time 5.620 (5.620)	 EPE 0.679 (0.679)
Test: [10/29]	 Time 0.297 (0.775)	 EPE 0.896 (3.263)
Test: [20/29]	 Time 0.241 (0.534)	 EPE 0.676 (3.334)
 * EPE 2.724
Epoch: [288]

Epoch: [293][80/102]	 Time 4.112 (0.997)	 Data 3.763 (0.732)	 Loss 17.456 (22.957)	 EPE 1.187 (1.781)
Epoch: [293][90/102]	 Time 0.237 (0.959)	 Data 0.000 (0.695)	 Loss 22.916 (23.009)	 EPE 1.406 (1.785)
Epoch: [293][100/102]	 Time 0.227 (0.920)	 Data 0.000 (0.658)	 Loss 19.123 (22.879)	 EPE 0.785 (1.755)
Test: [0/29]	 Time 5.563 (5.563)	 EPE 0.669 (0.669)
Test: [10/29]	 Time 0.288 (0.767)	 EPE 0.893 (3.201)
Test: [20/29]	 Time 0.238 (0.527)	 EPE 0.660 (3.342)
 * EPE 2.733
Epoch: [294][0/102]	 Time 21.224 (21.224)	 Data 20.891 (20.891)	 Loss 15.368 (15.368)	 EPE 0.606 (0.606)
Epoch: [294][10/102]	 Time 0.245 (2.519)	 Data 0.001 (2.252)	 Loss 21.713 (20.581)	 EPE 1.690 (1.799)
Epoch: [294][20/102]	 Time 0.246 (1.623)	 Data 0.001 (1.363)	 Loss 15.214 (19.975)	 EPE 0.957 (1.528)
Epoch: [294][30/102]	 Time 0.253 (1.309)	 Data 0.000 (1.048)	 Loss 34.671 (20.587)	 EPE 2.644 (1.498)
Epoch: [294][40/102]	 Time 4.139 (1.240)	 Data 3.783 (0.975)	 Loss 15.583 (21.381)	 EPE 0.893 (1.503)
Epoch: [2

Test: [20/29]	 Time 0.229 (0.585)	 EPE 0.669 (3.293)
 * EPE 2.691


In [1]:
%reload_ext tensorboard

In [2]:
%tensorboard --logdir "D:\My_Jupyter Notebook\PWC-Net-main\PyTorch\mpi_sintel_clean\11-27-21-25"